# Known Problem:
- **Unresolved**
    1. how to solve for eigen vectors
- **Partially Resolved**
    1. Certain websites cause the error of, specified here: https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests, which is a connection error. 
        - this appears to be windows specific as this was not a relevant issue when testing code in ubuntu 22.04, some forum reflect similar message
        - FAILED SOLUTION: first response in that stackoverflow page, still kept the stuff bc it seemed like better QOL
        - ATTEMPTED SOLUTION: used try and except. NOTE: this solution is not ideal as it does not resolve the issue, it just ignores it and moves on.
- **Resolved**
    1. The search "what is a taylor series" returns SSLCertVerification Error.
        - this resolution does not consider the security of the websites being accessed, potentially use try except to have the request verify when possible and not break when cannot verify
        - ATTEMPTED SOLUTION: used try except to use verify=False when ssl error occurs. Nested try except to ensure that both connection error and ssl error are handled.

    

    

# Part 1: Data Acquisition & Preparation

## Collecting & Preprocessing Our Data

In [1]:
#imports
import requests #send HTTP requests to web pages and retrieve their content
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from urllib.parse import urlparse, urljoin
import pandas as pd
from nltk.corpus import stopwords
import http.client
import re

In [2]:
import ssl
nltk.download('punkt')
#remove filler words from an example column
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/william/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/william/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/william/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import concurrent.futures
from tqdm import tqdm
#Working implementation

session = requests.Session()
retry = Retry(connect=2, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
# Define the search query and retrieve search results
search_query = input("What would you like to search up?")
search_engine_url = "https://www.google.com/search?q="

# Set headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Number of search result pages to retrieve (each page typically has 10 results)
num_pages = 5

all_urls = []
search_results = []

for page in range(num_pages):
    # Calculate the start index for the current page
    start_index = page * 10

    try:
        # Make a GET request to retrieve the search result page
        response = session.get(search_engine_url + search_query + f"&start={start_index}", headers=headers)
        response.raise_for_status()  # Raise an exception for non-2xx status codes

    except requests.exceptions.RequestException as e:
        print("Error occurred for page", page+1)
        print("Exception:", e)
        continue

    # Parse and extract information from the search result page
    soup = BeautifulSoup(response.content, "html.parser")

    for result in soup.select("div.g"):
        title_element = result.select_one("h3")
        if title_element:
            title = title_element.get_text()
            url_element = result.find("a")
            if url_element:
                url = url_element["href"]
                parsed_url = urlparse(url)
                if parsed_url.scheme == "":
                    # Relative URL, convert it to absolute URL
                    base_url = response.url
                    url = urljoin(base_url, url)
                all_urls.append(url)


def get_response(url):
    try:
        response = session.get(url, verify=True, headers=headers)
        result_soup = BeautifulSoup(response.content, "html.parser")
        text = result_soup.get_text()
        text = ' '.join(text.split())
        search_results.append({"Title": title, "URL": url, "Text": text})
    except http.client.IncompleteRead:
        print("Error: Incomplete read for:", url)
        return None
    except Exception as e:
        print("Error: Failed to retrieve content for:", url)
        print("Exception:", e)
        return None

executor = concurrent.futures.ThreadPoolExecutor(max_workers=10)

for cur_url in all_urls:
    futures = [
        executor.submit(
            get_response,
            cur_url
        )
    ]

results = []
with tqdm(total=len(futures), desc="Scraping...") as pbar:
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        results.append(results)
        pbar.update(1)

    
print("Grabbed search results")

# Process the search results as desired

Scraping...:   0%|          | 0/1 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Scraping...: 100%|██████████| 1/1 [00:04<00:00,  4.49s/it]

Grabbed search results


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [5]:
df = pd.DataFrame(search_results)
df_bart = pd.DataFrame(search_results)
df_bart_unlem = pd.DataFrame(search_results)
# Extract domain names from URLs and set as the index for the dataframe
tld_pattern = re.compile(r'\.[a-zA-Z]+$')
df['Domain'] = df['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df.insert(0, 'Domain', df.pop('Domain'))

df_bart['Domain'] = df_bart['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart.insert(0, 'Domain', df_bart.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart['Text'] = df_bart['Text'].str.replace('\ufffd', '')

df_bart_unlem['Domain'] = df_bart_unlem['URL'].apply(lambda url: re.sub(tld_pattern, '', urlparse(url).netloc.replace("www.", "")))
df_bart_unlem.insert(0, 'Domain', df_bart_unlem.pop('Domain'))
#Handles the "Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER" exception
df_bart_unlem['Text'] = df_bart_unlem['Text'].str.replace('\ufffd', '')

#unfiltered dataframe
print(len(df))
df.head()

45


,Domain,Title,URL,Text
0,statisticshowto,Complex Analysis Lecture Notes,https://www.statisticshowto.com/calculus-probl...,Complex Analysis - Statistics How To Skip to c...
1,complex-analysis,Complex Analysis Lecture Notes,https://complex-analysis.com/,Complex Analysis COMPLEX ANALYSIS A Visual and...
2,medium,Complex Analysis Lecture Notes,https://medium.com/analytics-vidhya/an-introdu...,An Introduction to Complex Analysis and Applic...
3,britannica,Complex Analysis Lecture Notes,https://www.britannica.com/science/complex-var...,Complex variable | mathematics | Britannica Se...
4,javiertordable,Complex Analysis Lecture Notes,https://www.javiertordable.com/beauty-complex-...,The Beauty of Complex Analysis – Javier Tordab...


In [6]:
stop_words = stopwords.words("english") #english filler words

def preprocess(text):
    '''
    Input a text block and filter out unneeded characters
    returns a filtered text block in the form of a str
    Function filters whitespace, numbers, special characters, stopwords; handles case normalization 
    '''
    # Remove special characters and numbers
    cleaned_text = re.sub(r"[^a-zA-Z]+", " ", text)
    
    # Convert to lowercase and split into words
    words = cleaned_text.lower().split()
    
    # Remove stop words and single-character words
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in filtered_words])
    
    return lemmatized_text

def bart_preprocess(text):

    words = text.split()
    lemmatized_text = " ".join([WordNetLemmatizer().lemmatize(word) for word in words])

    return lemmatized_text

In [7]:
#Properly using the apply() function onto a pandas column
#removes both filler words and special characters
df["Text"] = df["Text"].apply(preprocess)
df_bart["Text"] = df_bart["Text"].apply(bart_preprocess)


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# NOTE
Lemmatization doesn't seem to remove much here

In [32]:
#filtered text for first row entry
df_bart["Text"][0]

'How to Find Eigenvalues and Eigenvectors: 8 Steps (with Pictures) GuidesRANDOMLOG IN Log in Social login doe not work in incognito and private browsers. Please log in with your username or email to continue. Facebook Google wikiHow Account No account yet? Create an account EXPLORE CouponsTech Help ProRandom ArticleAbout Us QuizzesContributeTrain Your Brain GameThis Or That Game Popular Categories Arts and EntertainmentArtworkBooksMoviesComputers and ElectronicsComputersPhone SkillsTechnology HacksHealthMen\'s HealthMental HealthWomen\'s HealthRelationshipsDatingLoveRelationship Issues Hobbies and CraftsCraftsDrawingGamesEducation & CommunicationCommunication SkillsPersonal DevelopmentStudyingPersonal Care and StyleFashionHair CarePersonal HygieneYouthPersonal CareSchool StuffDating All Categories Arts and EntertainmentFinance and BusinessHome and GardenRelationship Quizzes Cars & Other VehiclesFood and EntertainingPersonal Care and StyleSports and Fitness Computers and ElectronicsHeal